In [2]:
import requests
import urllib
from bs4 import BeautifulSoup

In [3]:
# URL = "https://www.allrecipes.com/recipe/142377/delicious-feta-crusted-salmon/"
URL = "https://www.allrecipes.com/recipe/228141/marinated-fried-fish/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

## Get key containers

In [4]:
img = soup.find("div", {"class":"image-container"})
cal = soup.find("div", {"class":"recipe-nutrition-section"})

In [5]:
print(cal.prettify())

<div class="partial recipe-nutrition-section">
 <div class="section-label">
  Per Serving:
 </div>
 <div class="section-body">
  1345 calories; protein 25.3g; carbohydrates 37.4g; fat 123.5g; cholesterol 152.9mg; sodium 202.6mg.
  <a aria-label="Read Full Nutrition" class="recipe-nutrition-section-link link-view" data-tracking-do-not-track="1" href="#">
   Full Nutrition
  </a>
 </div>
</div>



In [6]:
print(img.prettify())

<div class="image-container">
 <div class="component lazy-image lazy-image-udf lead-media ugc-photos-link aspect_1x1" data-alt="Marinated Fried Fish " data-crop-percentage="100" data-high-density="false" data-main-recipe="true" data-original-height="2449" data-original-width="2448" data-shop-image="false" data-special-crop="" data-src="https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fimages.media-allrecipes.com%2Fuserphotos%2F1107016.jpg" data-title="" data-tracking-zone="image">
  <noscript>
   <img alt="Marinated Fried Fish " src="https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fimages.media-allrecipes.com%2Fuserphotos%2F1107016.jpg"/>
  </noscript>
  <div class="inner-container js-inner-container image-overlay">
   <span class="placeholderText visually-hidden">
    Marinated Fried Fish
   </span>
   <span class="icon icon-pinterest-circle-solid social-icon pinterest-transparent" data-social-service="pinterest">
    <a aria-label="Pin this Marinated Fried 

## Download Image

In [7]:
img_elements = img.find_all("div", class_="lazy-image")

In [8]:
img_url = img_elements[0]['data-src']

In [9]:
urllib.request.urlretrieve(img_url, "salmon.jpg")

('salmon.jpg', <http.client.HTTPMessage at 0x1efc29c5580>)

## Get Calorie Count

In [10]:
cal_text = cal.getText()
cal_text_list = cal_text.split(' ')

In [11]:
num_cal = int(cal_text_list[4])

In [12]:
num_cal

1345

## Function of above work

In [1]:
def get_data(search_term, url, recipe_num):
    page = requests.get(url)

    soup = BeautifulSoup(page.content, "html.parser")
    
    # Get Key Containers
    img = soup.find("div", {"class":"image-container"})
    cal = soup.find("div", {"class":"recipe-nutrition-section"})
    
    # Try Download Image
    try:
        img_elements = img.find_all("div", class_="lazy-image")
        if len(img_elements) > 0:
          
            img_url = img_elements[0]['data-src']
            urllib.request.urlretrieve(img_url, f"data/{search_term}/{search_term}_{recipe_num}.jpg")

        # Get Calorie Count
        cal_text = cal.getText()
        cal_text_list = cal_text.split(' ')
        num_cal = int(cal_text_list[4])
        with open(f'data/{search_term}_cal.csv', 'a+') as f:
            f.write(str(num_cal))
            f.write('\n')
    
    except:
        pass

## Make it automated

In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [15]:
import time

### Automate "fish" web scrape on allrecipes.com

In [66]:
def web_scrape(term):
    driver = webdriver.Firefox(executable_path=r'../geckodriver.exe')
    driver.get(f"https://www.allrecipes.com/search/results/?search={term}")
    assert term in driver.title

    driver.maximize_window() # For maximizing window

    # Reference: https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python
    SCROLL_PAUSE_TIME = .5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    num = 0

    for a in driver.find_elements_by_partial_link_text(term):
        if a.get_attribute("href") == "javascript:void(0);":
            pass
        else:
            get_data(term, a.get_attribute("href"), num)
            num = num+1

    driver.close()

In [72]:
web_scrape("Vegetable")

C:\Users\miahn\AppData\Local\Temp/ipykernel_5076/2208909105.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=r'../geckodriver.exe')
C:\Users\miahn\AppData\Local\Temp/ipykernel_5076/2208909105.py:29: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  for a in driver.find_elements_by_partial_link_text(term):
